In [306]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-
#__author__:yanbin
from pyspark import SparkContext,SparkConf
from operator import add,itemgetter
from pyspark.sql import SparkSession,Row
from pandas import *
from pyspark.sql.functions import *
from pyspark.ml.feature import Bucketizer
from pyspark.sql.types import *
import time
import math
class ItemCFModel(object):  
    def __init__(self,df,item_pair_sim=None,spark=None):
        self.df = df
        self.item_pair_sim = item_pair_sim

    def itemSimilarity(self):
        # RDD[(uid,(aid,score))] 
        user_item_score = self.df.rdd.map(lambda x:(x['user'],[x['item'],x['rating']]))
        item_score_pair = user_item_score.join(user_item_score)\
                        .map(lambda x:((x[1][0][0],x[1][1][0]),(x[1][0][1],x[1][1][1])))
        item_pair_ALL = item_score_pair.map(lambda f:(f[0], f[1][0] * f[1][1])).reduceByKey(add,300)
        item_pair_XX_YY = item_pair_ALL.filter(lambda f:f[0][0] == f[0][1])
        item_pair_XY = item_pair_ALL.filter(lambda f:f[0][0] != f[0][1])
        #RDD[(aid1,score11 * score11 + score21 * score21)] 
        item_XX_YY = item_pair_XX_YY.map(lambda f:(f[0][0], f[1]))
        #RDD(aid1,((aid1,aid2,XY),XX))
        item_XY_XX = item_pair_XY.map(lambda f:(f[0][0], (f[0][0], f[0][1], f[1]))).join(item_XX_YY) 
        #RDD[(aid2,((aid1,aid2,
        #           score11 * score12 + score21 * score22,score11 * score11 + score21 * score21),
        #           score12 * score12 + score22 * score22))] 
        item_XY_XX_YY = item_XY_XX.map(lambda f:(f[1][0][1],(f[1][0][0],f[1][0][1],f[1][0][2],f[1][1]))).join(item_XX_YY)  
        # item_XY_XX_YY中的(aid1,aid2,XY,XX,YY)) 
        # RDD[(aid1,aid2,
        # score11 * score12 + score21 * score22,score11 * score11 + score21 * score21,score12 * score12 + score22 * score22)]       
        item_pair_XY_XX_YY = item_XY_XX_YY.map(lambda f:(f[1][0][0], f[1][0][1], f[1][0][2], f[1][0][3], f[1][1]))  
        # item_pair_XY_XX_YY为(aid1,aid2,XY / math.sqrt(XX * YY)) 
        # RDD[(aid1,aid2,
        # score11 * score12 + score21 * score22 / math.sqrt((score11 * score11 + score21 * score21)*(score12 * score12 + score22 * score22))] 
        item_pair_sim = item_pair_XY_XX_YY.map(lambda f :(f[0], (f[1], f[2] / math.sqrt(f[3] * f[4]))))  
        return item_pair_sim
    

    def train(self):
        item_pair_sim = self.itemSimilarity()
        item_pair_sim.cache()  
        self.item_pair_sim=item_pair_sim
    



def recommend(df,item_pair_sim,NN=100,topN =10,Normalization=False):
    def itemNN(item_pair_sim,NN=100,Normalization=False):
        item_sim = item_pair_sim.filter(lambda f:f[1][1]>0.05)\
                            .groupByKey()\
                            .mapValues(list)
        if Normalization:
            def norm(x):
                m =  __builtin__.max([i[1] for i in x])
                l = []
                for i in x:
                    l.append((i[0],i[1]/m))
                return l
            item_sim = item_sim.mapValues(lambda x:norm(x))
        item_simNN = item_sim.mapValues(lambda x:sorted(x,key=itemgetter(1),reverse=True)[:NN])\
                            .collectAsMap()
        return item_simNN
    
    def getOrElse(f,item_sim_bd):
        items_sim = item_sim_bd.value.get(f[0][1]) 
        if items_sim is None:
            items_sim = [("0", 0.0)]
        for w in items_sim:
            yield ((f[0][0],w[0]),w[1]*f[1])
            
    user_item_score = df.rdd.map(lambda x:((x['user'],x['item']),x['rating']))
    item_sim_bd = sc.broadcast(itemNN(item_pair_sim,NN=NN,Normalization=Normalization))
#     /* 
#      * 提取item_sim_user_score为((user,item2),sim * score) 
#      * RDD[(user,item2),sim * score] 
#      */  

    user_item_simscore = user_item_score.flatMap(lambda f:getOrElse(f,item_sim_bd))\
                                        .filter(lambda f:f[1]> 0.03)  
#       /*
#      * 聚合user_item_simscore为 (user,（item2,sim1 * score1 + sim2 * score2）)
#      * 假设user观看过两个item,评分分别为score1和score2，item2是与user观看过的两个item相似的item,相似度分别为sim1，sim2 
#      * RDD[(user,item2),sim1 * score1 + sim2 * score2）)] 
#      */  
    user_item_rank = user_item_simscore.reduceByKey(add,1000)  

#     /* 
#      * 过滤用户已看过的item,并对user_item_rank基于user聚合 
#      * RDD[(user,CompactBuffer((item2,rank2）,(item3,rank3)...))] 
#      */  
    user_items_ranks = user_item_rank.subtractByKey(user_item_score)\
                                     .map(lambda f:(f[0][0], (f[0][1], f[1])))\
                                     .groupByKey()  
#     /* 
#      * 对user_items_ranks基于rank降序排序，并提取topN,其中包括用户已观看过的item 
#      * RDD[(user,ArrayBuffer((item2,rank2）,...,(itemN,rankN)))] 
#      */  
    user_items_ranks_desc = user_items_ranks.mapValues(list)\
                            .mapValues(lambda x:sorted(x,key=itemgetter(1),reverse=True)[:topN])
    return user_items_ranks_desc

from Evaluator import Evaluator
def evaluate(evaluator):
    precise = evaluator.precision()
    coverage = evaluator.coverage()
    popularity = evaluator.popularity()
    recall = evaluator.recall()
    return precise,recall,coverage,popularity

if __name__ == "__main__": 
    import os
    PYSPARK_PYTHON = "/usr/bin/python2.7"
    os.environ["PYSPARK_PYTHON"] = PYSPARK_PYTHON
    conf = SparkConf().setAppName("itemCF").setMaster("yarn")
    conf.set('spark.yarn.dist.files',
            'file:/root/hadoop-2.6/spark/python/lib/pyspark.zip,file:/root/hadoop-2.6/spark/python/lib/py4j-0.10.4-src.zip')
    conf.set('spark.executor.cores','30')
    conf.set('spark.executor.memory','95g')
    conf.set('spark.executor.instances','8')
    conf.set('spark.sql.shuffle.partitions','400')
    conf.set('spark.default.parallelism','200')
#     conf.set("spark.shuffle.file.buffer","128k").set("spark.reducer.maxSizeInFlight","96M")
    spark = SparkSession.builder\
        .config(conf=conf) \
        .getOrCreate()
    sc=spark.sparkContext 
    sc.setLogLevel('WARN')
    start = time.time()
    inputPath = 'data/ml-1m/ratings.dat'
    schema = StructType([
            StructField("user", StringType(), True),
            StructField("item", StringType(), True),
            StructField("rating", DoubleType(), True),
            StructField("timestamp", LongType(), True)])
#     ratingRdd = sc.textFile(inputPath).map(lambda line:line.split("::"))\
#                     .map(lambda x:(x[0],x[1],float(x[2]),long(x[3])))
#     ratingDf = spark.createDataFrame(data=ratingRdd,schema=schema)
    #     ratingDf,_ = ratingDf.randomSplit([1.0,9.0],seed=40)
    ratingDf = spark.read.parquet('data/ml-1m/user_parquet')
    ratingDf = ratingDf.repartition(300)
    ratingDf.printSchema()
    ratingDf.show(5)
    n = ratingDf.count()
#     ratingDf = ratingDf.withColumn('score',col('rating')*0+1).select('user','item','score')    
    print 'total lines: %s' %n
    train,test = ratingDf.randomSplit([4.0,1.0],seed=40)
    train.cache()
    test.cache()
    itemCF = ItemCFModel(df=train,spark=spark)
    itemCF.train()
#     for NN in [5,10,20,40,60,80,100,120]:
    for NN in [100]:
        recTopN = recommend(train,itemCF.item_pair_sim,NN=NN,topN=10,Normalization=False)
        pre = spark.createDataFrame(data=recTopN.flatMapValues(lambda x:x).map(lambda x:(x[0],x[1][0],x[1][1])),
                                    schema=['user','item','rating'])
        evaluator = Evaluator(train,test,pre)
        (precise,recall,coverage,popularity) = evaluate(evaluator)
        end = time.time()
        print ('NN:%3d,  precise:%2.2f%%,  recall:%2.2f%%,  coverage:%2.2f%%,  popularity:%2.2f, time: %s s'
                %(NN,precise*100,recall*100,coverage*100,popularity,end-start))
        pre.write.parquet(partitionBy='user',path='data/ml-1m/result/itemCF/itcf_'+str(NN),mode='ignore')
        start = time.time()

root
 |-- item: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- user: integer (nullable = true)

+----+------+---------+----+
|item|rating|timestamp|user|
+----+------+---------+----+
|2902|   4.0|971581445|4169|
|1480|   3.0|973310869|4169|
|2521|   2.0|974956577|1941|
|3421|   5.0|970049117|4277|
|3763|   4.0|974715698|1680|
+----+------+---------+----+
only showing top 5 rows

total lines: 1000209


In [227]:
start = time.time()
recTopN = recommend(train,itemCF.item_pair_sim,NN=NN,topN=100,Normalization=False)
pre = spark.createDataFrame(data=recTopN.flatMapValues(lambda x:x).map(lambda x:(x[0],x[1][0],x[1][1])),
                            schema=['user','item','rating'])
evaluator = Evaluator(train,test,pre)
(precise,recall,coverage,popularity) = evaluate(evaluator)
end = time.time()
print ('NN:%3d,  precise:%2.2f%%,  recall:%2.2f%%,  coverage:%2.2f%%,  popularity:%2.2f, time: %s s'
        %(NN,precise*100,recall*100,coverage*100,popularity,end-start))


NN:100,  precise:12.40%,  recall:37.42%,  coverage:37.16%,  popularity:6.97, time: 80.6126019955 s


In [161]:
from pyspark.ml.feature import StandardScaler,VectorAssembler,MaxAbsScaler
from pyspark.ml.linalg import Vectors
normalizer = Normalizer(p=2.0, inputCol="vector", outputCol="new")
vecAssembler = VectorAssembler(inputCols=["rating"], outputCol="vector")
maScaler = MaxAbsScaler(inputCol="vector", outputCol="scaled")
standardScaler = StandardScaler(inputCol="vector", outputCol="scaled",withMean=True,withStd=False)
from pyspark.ml import Pipeline,PipelineModel
pipe = Pipeline(stages=[vecAssembler,standardScaler])
scaledPre = pipe.fit(pre).transform(pre)
# pre.filter('user=1').withColumn('new',col('rating')*max(col('rating'))+1.0)

In [1]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-
#__author__:yanbin
from pyspark import SparkContext,SparkConf
from operator import add,itemgetter
from pyspark.sql import SparkSession,Row
from pyspark.ml.linalg import Vector,Vectors
from pandas import *
from pyspark.sql.functions import *
from pyspark.ml.feature import Bucketizer
from pyspark.sql.types import *
import time,math,os,re
PYSPARK_PYTHON = "/usr/bin/python2.7"
os.environ["PYSPARK_PYTHON"] = PYSPARK_PYTHON
conf = SparkConf().setAppName("itemCF").setMaster("yarn")
conf.set('spark.yarn.dist.files',
        'file:/root/hadoop-2.6/spark/python/lib/pyspark.zip,file:/root/hadoop-2.6/spark/python/lib/py4j-0.10.4-src.zip')
conf.set('spark.executor.cores','30')
conf.set('spark.executor.memory','95g')
conf.set('spark.executor.instances','8')
conf.set('spark.sql.shuffle.partitions','400')
conf.set('spark.default.parallelism','200')
conf.set('spark.driver.maxResultSize', '10g')
conf.set('spark.scheduler.mode','fair')

#     conf.set("spark.shuffle.file.buffer","128k").set("spark.reducer.maxSizeInFlight","96M")
spark = SparkSession.builder\
    .config(conf=conf) \
    .getOrCreate()
sc=spark.sparkContext 
sc.setLogLevel('WARN')
sc.setLocalProperty("spark.scheduler.pool", "rank")
start = time.time()
ratingsPath = 'data/ml-1m/ratings.dat'
moviesPath = 'data/ml-1m/movies.dat'
usersPath = 'data/ml-1m/users.dat'

rating_schema = StructType([
                StructField("userId", StringType(), True),
                StructField("itemId", StringType(), True),
                StructField("rating", DoubleType(), True),
                StructField("timestamp", LongType(), True)])

movie_schema = StructType([
                StructField("itemId", StringType(), True),
                StructField("title", IntegerType(), True),
                StructField("genres", ArrayType(FloatType()), True)])

user_schema = StructType([
                StructField("userId", StringType(), True),
                StructField("gender", StringType(), True),
                StructField("age", IntegerType(), True),
                StructField("occupation", IntegerType(), True),
                StructField("zip-code", StringType(), True)])
genresDict ={
 'Action': 0,
 'Adventure': 1,
 'Animation': 2,
 "Children's": 3,
 'Comedy': 4,
 'Crime': 5,
 'Documentary': 6,
 'Drama': 7,
 'Fantasy': 8,
 'Film-Noir': 9,
 'Horror': 10,
 'Musical': 11,
 'Mystery': 12,
 'Romance': 13,
 'Sci-Fi': 14,
 'Thriller': 15,
 'War': 16,
 'Western': 17}
#rating 数据DataFrame获取
ratingRdd = sc.textFile(ratingsPath).map(lambda line:line.split("::"))\
                .map(lambda x:(x[0],x[1],float(x[2]),long(x[3])))
ratingDf = spark.createDataFrame(data=ratingRdd,schema=rating_schema)
#movie 属性DataFrame获取


def threeTuple(x,genresDict):
    arr = np.zeros((18))
    for k in x.split('|'):
        i = genresDict.get(k)
        if  i is not None:arr[i] = 1
    return arr.tolist()
movieRdd = sc.textFile(moviesPath).map(lambda line:line.split("::"))\
                .map(lambda x:(x[0],int(re.findall(r'[^()]+', x[1])[-1]),Vectors.dense(threeTuple(x[2],genresDict))))
movieDf = spark.createDataFrame(data=movieRdd,schema=['itemId','title','genres'])
#user 属性DataFrame获取
userRdd = sc.textFile(usersPath).map(lambda line:line.split("::"))\
                .map(lambda x:(x[0],x[1],int(x[2]),int(x[3]),x[4]))
userDf = spark.createDataFrame(data=userRdd,schema=user_schema)
#     ratingDf,_ = ratingDf.randomSplit([1.0,9.0],seed=40)

ratingDf.show(5)
movieDf.show(5)
userDf.show(5)
userIds = ratingDf.select('userId').distinct().rdd.map(lambda x:x['userId']).collect()
# userIds = [u'1']
posDf = spark.createDataFrame(data=[],schema=rating_schema)
negDf = spark.createDataFrame(data=[],schema=rating_schema)

for userId in userIds:
    pos = ratingDf.filter('userId='+userId)
    n = pos.count()
    posDf = posDf.unionAll(pos)
    negDf = negDf.unionAll(ratingDf.filter('userId!='+userId).sample(False,0.1).limit(n))

posDf.cache()
negDf.cache()
posDf.show(5)
negDf.show(5)
df = posDf.unionAll(negDf)
# posDf = posDf.join(movieDf,on='itemId').join(userDf,on='userId')
# posDf = posDf.withColumn('label',col('rating')*0)
# negDf = negDf.join(movieDf,on='itemId').join(userDf,on='userId')
# negDf = negDf.withColumn('label',col('rating')*0+1.0).withColumn('label',col('rating')*0+1.0)

df.show(5)    
print '训练数据集：',df.count()

+------+------+------+---------+
|userId|itemId|rating|timestamp|
+------+------+------+---------+
|     1|  1193|   5.0|978300760|
|     1|   661|   3.0|978302109|
|     1|   914|   3.0|978301968|
|     1|  3408|   4.0|978300275|
|     1|  2355|   5.0|978824291|
+------+------+------+---------+
only showing top 5 rows

+------+-----+--------------------+
|itemId|title|              genres|
+------+-----+--------------------+
|     1| 1995|[0.0,0.0,1.0,1.0,...|
|     2| 1995|[0.0,1.0,0.0,1.0,...|
|     3| 1995|[0.0,0.0,0.0,0.0,...|
|     4| 1995|[0.0,0.0,0.0,0.0,...|
|     5| 1995|[0.0,0.0,0.0,0.0,...|
+------+-----+--------------------+
only showing top 5 rows

+------+------+---+----------+--------+
|userId|gender|age|occupation|zip-code|
+------+------+---+----------+--------+
|     1|     F|  1|        10|   48067|
|     2|     M| 56|        16|   70072|
|     3|     M| 25|        15|   55117|
|     4|     M| 45|         7|   02460|
|     5|     M| 25|        20|   55455|
+------+-

Py4JJavaError: An error occurred while calling o36418.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Total size of serialized results of 8289 tasks (10.0 GB) is bigger than spark.driver.maxResultSize (10.0 GB)
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1435)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1423)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1422)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1422)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:802)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGS

In [6]:
posDf.printSchema()

root
 |-- userId: string (nullable = true)
 |-- itemId: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- title: long (nullable = true)
 |-- genres: vector (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- occupation: integer (nullable = true)
 |-- zip-code: string (nullable = true)
 |-- label: double (nullable = true)



In [309]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression,LogisticRegressionModel
from pyspark.ml.classification import BinaryLogisticRegressionSummary
from pyspark.ml.feature import StringIndexer,IndexToString,HashingTF,Tokenizer,VectorIndexer,VectorAssembler
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline,PipelineModel
labelInder = StringIndexer(inputCol='label',outputCol='indexedLabel').fit(df)
genderInder = StringIndexer(inputCol='gender',outputCol='indexedGender').fit(df)
zipInder = StringIndexer(inputCol='zip-code',outputCol='indexedZip').fit(df)
normalizer = Normalizer(p=2.0, inputCol="vector", outputCol="new")
vecAssembler1 = VectorAssembler(inputCols=['timestamp','title','age','indexedGender','occupation','indexedZip']
                               , outputCol="feature1")
vecAssembler2 = VectorAssembler(inputCols=['feature1','genres']
                               ,outputCol="features")

lr = LogisticRegression(featuresCol='features',labelCol='label',\
                        maxIter=10,regParam=2,elasticNetParam=0.8)
# featureIndex = VectorIndexer(inputCol='features',outputCol='indexedFeatures').fit(df)
(traningDf,testDf) = df.randomSplit(seed=40,weights=[0.7,0.3])
# labelConverter = IndexToString(inputCol='prediction',outputCol='predictedLabel',labels=labelInder.labels)
lrPipeline = Pipeline().setStages([labelInder,genderInder,zipInder,vecAssembler1,vecAssembler2,lr])
lrModel = lrPipeline.fit(traningDf)
lrPredictions = lrModel.transform(testDf)

AnalysisException: u"cannot resolve '`userId`' given input columns: [item, rating, timestamp, user]; line 1 pos 0;\n'Filter ('userId = 1)\n+- Repartition 300, true\n   +- Relation[item#40501,rating#40502,timestamp#40503L,user#40504] parquet\n"

##  用户候选集选出Top100，经排序选出Top10，效果对比
### Top100排评分选出Top10

In [233]:
top10 = pre.filter('user=1').sort(desc('rating')).limit(10)
top10.show()

+----+----+------------------+
|user|item|            rating|
+----+----+------------------+
|   1|1196| 48.49930907578779|
|   1|1270| 47.61567828646564|
|   1|1198| 45.08401892251104|
|   1| 919| 44.20005863231689|
|   1|1210| 42.93014370986168|
|   1|1197|41.348620242209684|
|   1|1580| 39.99374911460752|
|   1|2716|37.961563721223165|
|   1| 593|36.941247196018764|
|   1|1265| 36.74054918535605|
+----+----+------------------+



In [232]:
evaluator = Evaluator(train,test.filter('user=1'),top10)
(precise,recall,coverage,popularity) = evaluate(evaluator)
print ('NN:%3d,  precise:%2.2f%%,  recall:%2.2f%%,  coverage:%2.2f%%,  popularity:%2.2f, time: %s s'
        %(NN,precise*100,recall*100,coverage*100,popularity,end-start))

NN:100,  precise:30.00%,  recall:23.08%,  coverage:0.27%,  popularity:7.58, time: 80.6126019955 s


In [307]:

preDf = pre.filter('user=1').join(movieDf,pre.item == movieDf.itemId,'inner')
preDf = preDf.join(userDf,userDf.userId == preDf.user,'inner')
preDf.show(5)
lrPredictions = lrModel.transform(preDf)
exVec = udf(lambda x:x.values.tolist()[1],DoubleType())
lrPredictions = lrPredictions.select('user','item',exVec('probability').alias('probability'),'prediction')
top10_new = lrPredictions.sort(desc('probability')).limit(10)
evaluator = Evaluator(train,test.filter('user=1'),top10_new)
(precise,recall,coverage,popularity) = evaluate(evaluator)
print ('NN:%3d,  precise:%2.2f%%,  recall:%2.2f%%,  coverage:%2.2f%%,  popularity:%2.2f, time: %s s'
        %(NN,precise*100,recall*100,coverage*100,popularity,end-start))

+----+----+------------------+------+-----+--------------------+------+------+---+----------+--------+
|user|item|            rating|itemId|title|              genres|userId|gender|age|occupation|zip-code|
+----+----+------------------+------+-----+--------------------+------+------+---+----------+--------+
|   1|2294|17.304174172598724|  2294| 1998|[0.0,0.0,1.0,1.0,...|     1|     F|  1|        10|   48067|
|   1| 296| 34.08935371931486|   296| 1994|[0.0,0.0,0.0,0.0,...|     1|     F|  1|        10|   48067|
|   1|1394|23.186440705224598|  1394| 1987|[0.0,0.0,0.0,0.0,...|     1|     F|  1|        10|   48067|
|   1|1500|14.077631919065606|  1500| 1997|[0.0,0.0,0.0,0.0,...|     1|     F|  1|        10|   48067|
|   1|2628| 18.93425303048274|  2628| 1999|[1.0,1.0,0.0,0.0,...|     1|     F|  1|        10|   48067|
+----+----+------------------+------+-----+--------------------+------+------+---+----------+--------+
only showing top 5 rows

NN:100,  precise:10.00%,  recall:7.69%,  coverag

In [308]:
print test.filter('user=1').select('item').collect()
lrPredictions.sort(desc('probability')).show(100)

[Row(item=u'1270'), Row(item=u'1545'), Row(item=u'2294'), Row(item=u'1961'), Row(item=u'1029'), Row(item=u'1246'), Row(item=u'919'), Row(item=u'527'), Row(item=u'745'), Row(item=u'783'), Row(item=u'1193'), Row(item=u'1197'), Row(item=u'3105')]
+----+----+------------------+----------+
|user|item|       probability|prediction|
+----+----+------------------+----------+
|   1|2294|0.5125000000000001|       1.0|
|   1| 296|0.5125000000000001|       1.0|
|   1|1394|0.5125000000000001|       1.0|
|   1|1500|0.5125000000000001|       1.0|
|   1|2628|0.5125000000000001|       1.0|
|   1|1259|0.5125000000000001|       1.0|
|   1|1584|0.5125000000000001|       1.0|
|   1|1032|0.5125000000000001|       1.0|
|   1|1136|0.5125000000000001|       1.0|
|   1|  34|0.5125000000000001|       1.0|
|   1|2000|0.5125000000000001|       1.0|
|   1|2396|0.5125000000000001|       1.0|
|   1|1278|0.5125000000000001|       1.0|
|   1|1387|0.5125000000000001|       1.0|
|   1|2174|0.5125000000000001|       1.0|


In [7]:
sc.stop()
spark.stop()

In [109]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression,LogisticRegressionModel
from pyspark.ml.classification import BinaryLogisticRegressionSummary
mEvaluator = MulticlassClassificationEvaluator(labelCol='label',predictionCol='prediction')
mlrAccuracy = mEvaluator.evaluate(lrPredictions)
mlrModel = lrModel
print "Coefficients: " , mlrModel.coefficientMatrix.toArray(),"\n"\
        "Intercept: ",mlrModel.interceptVector,"\n"\
        "numClasses: ",mlrModel.numClasses,"\n"\
        "numFeatures: ",mlrModel.numFeatures
mlrAccuracy = mEvaluator.evaluate(lrPredictions)
print ('Test Error = ',1-mlrAccuracy)


Coefficients:  [[ 0.          0.00089466 -0.04180277  0.1691317  -0.00161059  0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]] 
Intercept:  [0.000152405445756] 
numClasses:  2 
numFeatures:  24
('Test Error = ', 0.0)
